In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os

In [2]:
# Define o diretório base como a pasta do projeto
BASE_DIR = os.getcwd()


DATA_PATH = os.path.join(BASE_DIR, '..', 'data', 'users_behavior.csv')


df = pd.read_csv(DATA_PATH )

In [3]:
print(df.head())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [6]:
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=54321)

Usei test_size=0.2 para separar 20% dos dados para o conjunto de teste. Isso deixa 80% dos dados originais para serem divididos ainda mais entre treinamento e validação.

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(
   features, target, test_size=0.25, random_state=12345) 

Pelo objetivo ser dividir os 80% de dados restantes em treinamento e validação. Se queremos que a validação seja 20% do total de dados, ela deve ser 25% dos 80% restantes (porque 25% de 80% é igual a 20% do total).

In [8]:
model = RandomForestClassifier(random_state=54321)
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
print(f"Acurácia no conjunto de teste: {accuracy_test}")

Acurácia no conjunto de teste: 0.942457231726283


In [9]:
# Inicializar o modelo de Regressão Logística
logistic_model = LogisticRegression(random_state=54321)

# Treinar o modelo
logistic_model.fit(features_train, target_train)

# Fazer previsões no conjunto de teste
predictions_test = logistic_model.predict(features_test)

# Avaliar a acurácia do modelo no conjunto de teste
accuracy_test = accuracy_score(target_test, predictions_test)
print(f"Acurácia no conjunto de teste: {accuracy_test}")

Acurácia no conjunto de teste: 0.7076205287713841


In [10]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
print(f"Acurácia no conjunto de teste: {accuracy_test}")

Acurácia no conjunto de teste: 0.9471228615863142


In [11]:
# Verificando se a acurácia no conjunto de teste é maior que 0,75
if accuracy_test > 0.75:
    print("O modelo atende ao limite de acurácia estabelecido pelo projeto.")
else:
    print("O modelo não atende ao limite de acurácia. Considere ajustar ou trocar o modelo e/ou hiperparâmetros.")

O modelo atende ao limite de acurácia estabelecido pelo projeto.


In [12]:
# Inicializar o modelo de Regressão Logística
logistic_model = LogisticRegression(random_state=12345)

# Treinar o modelo
logistic_model.fit(features_train, target_train)

# Fazer previsões no conjunto de teste
predictions_test = logistic_model.predict(features_test)

# Avaliar a acurácia do modelo no conjunto de teste
accuracy_test = accuracy_score(target_test, predictions_test)
print(f"Acurácia no conjunto de teste: {accuracy_test}")

Acurácia no conjunto de teste: 0.7076205287713841


In [13]:
# Verificando se a acurácia no conjunto de teste é maior que 0,75
if accuracy_test > 0.75:
    print("O modelo atende ao limite de acurácia estabelecido pelo projeto.")
else:
    print("O modelo não atende ao limite de acurácia. Considere ajustar ou trocar o modelo e/ou hiperparâmetros.")

O modelo não atende ao limite de acurácia. Considere ajustar ou trocar o modelo e/ou hiperparâmetros.


Fica evidente que com a mudança do modelo houve uma brusca mudança na acurácia, tendo, desta forma, o modelo Regressão Logística sido rejeitado.

In [14]:
# Modelo 1: RandomForestClassifier
# Definir o espaço de hiperparâmetros para a busca em grade
param_grid_forest = {
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializar GridSearchCV
grid_search_forest = GridSearchCV(
    RandomForestClassifier(random_state=54321),
    param_grid_forest,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Executar a busca em grade
grid_search_forest.fit(features_train, target_train)

# Recuperar o melhor modelo
best_forest_model = grid_search_forest.best_estimator_

# Avaliar o melhor modelo no conjunto de validação
forest_predictions_valid = best_forest_model.predict(features_valid)
forest_accuracy_valid = accuracy_score(target_valid, forest_predictions_valid)

# Imprimir os melhores hiperparâmetros e a acurácia no conjunto de validação
print(f"Random Forest Melhores Hiperparâmetros: {grid_search_forest.best_params_}")
print(f"Random Forest Acurácia no conjunto de validação: {forest_accuracy_valid}")

# Avaliar o melhor modelo no conjunto de teste
forest_predictions_test = best_forest_model.predict(features_test)
forest_accuracy_test = accuracy_score(target_test, forest_predictions_test)

# Imprimir a acurácia no conjunto de teste
print(f"Random Forest Acurácia no conjunto de teste: {forest_accuracy_test}")

Random Forest Melhores Hiperparâmetros: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Random Forest Acurácia no conjunto de validação: 0.8097014925373134
Random Forest Acurácia no conjunto de teste: 0.8429237947122862


In [15]:
# Modelo 2: LogisticRegression

# Crie um pipeline que primeiro padroniza os dados e depois aplica a regressão logística
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=54321))


# Defina o espaço de hiperparâmetros, incluindo o 'C'
param_grid_logistic = {
    'logisticregression__C': [0.01, 0.1, 1, 10, 100],
    'logisticregression__solver': ['liblinear', 'saga']
}

# Inicialize o GridSearchCV com o pipeline
grid_search_logistic = GridSearchCV(
    pipeline,
    param_grid_logistic,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Execute a busca em grade
grid_search_logistic.fit(features_train, target_train)

# Recuperar o melhor modelo
best_logistic_model = grid_search_logistic.best_estimator_

# Avaliar o melhor modelo no conjunto de validação
logistic_predictions_valid = best_logistic_model.predict(features_valid)
logistic_accuracy_valid = accuracy_score(target_valid, logistic_predictions_valid)

# Imprimir os melhores hiperparâmetros e a acurácia no conjunto de validação
print(f"Logistic Regression Melhores Hiperparâmetros: {grid_search_logistic.best_params_}")
print(f"Logistic Regression Acurácia no conjunto de validação: {logistic_accuracy_valid}")

# Avaliar o melhor modelo no conjunto de teste
logistic_predictions_test = best_logistic_model.predict(features_test)
logistic_accuracy_test = accuracy_score(target_test, logistic_predictions_test)

# Imprimir a acurácia no conjunto de teste
print(f"Logistic Regression Acurácia no conjunto de teste: {logistic_accuracy_test}")

Logistic Regression Melhores Hiperparâmetros: {'logisticregression__C': 0.01, 'logisticregression__solver': 'liblinear'}
Logistic Regression Acurácia no conjunto de validação: 0.7611940298507462
Logistic Regression Acurácia no conjunto de teste: 0.7060653188180405


Apesar de não ter visto está função no modulo atual(a função GridSearchCV), achei de suma importancia seu uso para otimizar o modelo de aprendizado de máquina, pois ele automatiza o processo de experimentar várias combinações de hiperparâmetros, avaliando cada uma delas usando validação cruzada. Dessa forma, evitando o uso de força bruta.

*Conclusões:*

Primeiro modelo (Random Forest Classifier):

Melhores Hiperparâmetros: O GridSearchCV identificou que a combinação ideal de hiperparâmetros para o RandomForestClassifier é 100 árvores (n_estimators), profundidade máxima de 10 (max_depth), mínimo de uma amostra por folha (min_samples_leaf), e mínimo de duas amostras para divisão de um nó (min_samples_split).
Acurácia no Conjunto de Validação: A acurácia de aproximadamente 80.97% no conjunto de validação indica que o modelo é confiável e tem um bom desempenho em dados que não foram usados durante o treinamento.
Acurácia no Conjunto de Teste: A acurácia de aproximadamente 84.29% no conjunto de teste mostra que o modelo mantém um bom desempenho em dados não vistos, indicando uma boa capacidade de generalização.
Conclusão: O modelo Random Forest Classifier continua sendo um modelo adequado para a tarefa, atendendo aos requisitos do projeto e exibindo boa generalização. A diferença entre a acurácia no conjunto de validação e teste é pequena, o que sugere que não houve sobreajuste significativo.

Segundo modelo (Logistic Regression):

Melhores Hiperparâmetros: Para a LogisticRegression, o GridSearchCV encontrou que um valor de C de 0.01 com o solver 'liblinear' são os melhores hiperparâmetros.
Acurácia no Conjunto de Validação: A acurácia de aproximadamente 76.12% na validação sugere uma performance razoável, mas ainda há espaço para melhorias, especialmente se comparada com a Random Forest.
Acurácia no Conjunto de Teste: A acurácia de aproximadamente 70.61% no conjunto de teste está abaixo do limiar de 75% estabelecido pelo projeto, indicando que o modelo pode não ser o mais adequado para esta tarefa específica sem mais ajustes.
Conclusão: A Regressão Logística, embora seja um modelo mais simples e rápido de treinar, apresentou uma acurácia inferior ao modelo de Random Forest tanto na validação quanto no teste, o que pode indicar uma menor capacidade de capturar a complexidade dos dados. O baixo valor de C implica em uma forte regularização, o que pode ser necessário para evitar o sobreajuste, mas também pode estar limitando a capacidade do modelo de se ajustar aos dados.

*Conclusão geral:* 
Comparando ambos os modelos, o Random Forest Classifier é superior em termos de acurácia tanto no conjunto de validação quanto no de teste, sugerindo que é um modelo mais robusto para este conjunto de dados. A Regressão Logística pode ser útil para uma interpretação mais direta dos efeitos das características e para casos onde a velocidade de treinamento e predição é crítica, mas pode requerer uma engenharia de características mais cuidadosa e potencialmente mais dados para alcançar uma performance comparável à da Random Forest neste projeto específico.





